In [1]:
# For making plots for the paper
import matplotlib.pyplot as plt
plt.style.use("seaborn-poster")
plt.rcParams["figure.figsize"] = (10, 6) # 5/3

In [2]:
N = 128
NP = 3
NCORES = 1
P = NP * NCORES


lower_bounds = []
upper_bounds = []
ub_diff_lb_greater_than_one = []

# In some cases the result of the mulplication is a bit strange...
# example, if N//P is always 1, how is it that a range like (30, 32) is possible?
for rank in range(P):
    lb = (rank)*(N//P)  # This gives the intended loads where the last load is unbalanced
    ub = (rank+1)*(N//P) if rank != P-1 else N
    lower_bounds.append(lb)
    upper_bounds.append(ub)

    if ub - lb > 1:
        print(rank, lb, ub)
        ub_diff_lb_greater_than_one.append((lb, ub))

ub_diff_lb_greater_than_one

0 0 42
1 42 84
2 84 128


[(0, 42), (42, 84), (84, 128)]

In [29]:
# This is the most evenly load balanced data but how can this be determiend programmatically
N = 128 

# Most load balanced sizes
def load_balanced_sizes(N, P):
    sizes = [N//P for p in range(P)]
    for rank in range(N%P):
        sizes[rank] += 1
    return sizes 

for NP in range(1, 9):
    for NCORES in [1, 4, 16]:
        P = NP * NCORES
        sizes = load_balanced_sizes(N, P)
        assert sum(sizes) == N
        print(sizes)


[128]
[32, 32, 32, 32]
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[64, 64]
[16, 16, 16, 16, 16, 16, 16, 16]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[43, 43, 42]
[11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[32, 32, 32, 32]
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[26, 26, 26, 25, 25]
[7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
# Given the sizes (which is the difference between ub and lb)
# compute the lb and ub for the processes
N = 128
P = 3 

# Displacements are cum sum of recvcounts (sizes)
sizes = load_balanced_sizes(N=N, P=P)

def get_displacements_from_recvcounts(recvcounts):
    displacements = [0 for p in range(len(recvcounts))]
    for rank in range(1, P):
        displacements[rank] = displacements[rank-1] + sizes[rank]
    return displacements

displacements = get_displacements_from_recvcounts(recvcounts=sizes)

assert sum(displacements) == N

[43, 43, 42]


In [3]:
for p in zip(lower_bounds, upper_bounds):
    print(p)

(0, 42)
(42, 84)
(84, 128)


In [4]:
N//P

42